In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss, BCELoss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
#!pip install torchvision
#!pip install opencv-python

In [3]:
#training controls
batch_size = 32
epochs = 20
training_size = 0.8
learning_rate = 0.0003

# input image dimensions
img_rows, img_cols = 268, 182

In [4]:
# the data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [5]:
#opening the dataset
dataset = csv.reader(open("MovieGenre.csv",encoding="utf8",errors='replace'), delimiter=",")

#skipping the header line
next(dataset)

['\ufeffimdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [6]:
import zipfile as zf
files = zf.ZipFile("SampleMoviePosters.zip", 'r')
files.extractall()
files.close()

In [7]:
#the list of image files in SampleMoviePosters folder
flist=glob.glob('SampleMoviePosters/*.jpg')  

In [8]:
len(flist)

997

In [9]:
image_ids = []

for path in flist:
    start = path.rfind("\\")+1
    end = len(path)-4
    image_ids.append(path[start:end])
    
#image_ids

In [10]:
import pandas as pd

dataset2 = pd.read_csv("MovieGenre.csv")
dataset2

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...
40103,83168,http://www.imdb.com/title/tt83168,Tanya's Island (1980),4.3,Drama,https://images-na.ssl-images-amazon.com/images...
40104,82875,http://www.imdb.com/title/tt82875,Pacific Banana (1981),4.7,Comedy,https://images-na.ssl-images-amazon.com/images...
40105,815258,http://www.imdb.com/title/tt815258,Werewolf in a Womens Prison (2006),4.5,Horror,https://images-na.ssl-images-amazon.com/images...
40106,79142,http://www.imdb.com/title/tt79142,Xiao zi ming da (1979),6.5,Action|Comedy,https://images-na.ssl-images-amazon.com/images...


In [11]:
y = []
indexlist = []
classes = tuple()
ids = dataset2.imdbId.values.tolist()
for image_id in image_ids:
    genres = tuple((dataset2[dataset2["imdbId"] == int(image_id)]["Genre"].values[0]).split("|"))
    if int(image_id) in ids:
        indexlist.append(image_id)
    y.append(genres)
    classes = classes + genres
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)
classes = set(classes)
classes = list(classes)
classes.sort()

In [12]:
y_df = pd.DataFrame(y, columns = classes, index = indexlist)
y_df

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Music,Musical,Mystery,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
10040,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10057,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10071,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10155,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10195,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9899,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9932,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
9937,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
9968,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [13]:
#y_df_reset = y_df.reset_index()

#y_df_reset["index"] = pd.to_numeric(y_df_reset["index"])

#result = pd.merge(dataset2, y_df_reset, left_on='imdbId', right_on='index')

#collist = result.columns.tolist()
#collist.remove('Imdb Link')
#collist.remove('IMDB Score')
#collist.remove('index')
#result = result[collist]

#result

In [14]:
y_df_reset = y_df.reset_index()

shape = y_df_reset.shape[1]

index_value = []
genre_lst = []

for i in range(len(y_df_reset)):
    index_value.append(int(y_df_reset.loc[i,"index"]))
    temp_list = []
    for j in y_df_reset.columns[1:]:
        temp_list.append(y_df_reset.loc[i,j])
    genre_lst.append(temp_list)

df = pd.DataFrame(list(zip(index_value, genre_lst)),
               columns =['imdbId', 'genrelst'])

result = dataset2.merge(df, on="imdbId")
result

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,genrelst
0,24252,http://www.imdb.com/title/tt24252,Liebelei (1933),7.7,Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,25316,http://www.imdb.com/title/tt25316,It Happened One Night (1934),8.2,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,25164,http://www.imdb.com/title/tt25164,The Gay Divorcee (1934),7.6,Comedy|Musical|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,17350,http://www.imdb.com/title/tt17350,The Scarlet Letter (1926),7.8,Drama,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,25586,http://www.imdb.com/title/tt25586,Of Human Bondage (1934),7.3,Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
1013,23797,http://www.imdb.com/title/tt23797,Betty Boop's Big Boss (1933),6.6,Animation|Short|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1014,22670,http://www.imdb.com/title/tt22670,Betty Boop's Museum (1932),6.9,Animation|Short|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1015,19640,http://www.imdb.com/title/tt19640,Angora Love (1929),7.0,Comedy|Short,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1016,24676,http://www.imdb.com/title/tt24676,TÌ«kyÌ« no onna (1933),7.2,Drama,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
for x in range(len(result)):
    tempY.append((int(result['imdbId'].iloc[x]),result['genrelst'].iloc[x]))

#tempY

In [16]:
#setting the length of training data
length=int(len(flist)*training_size)
length

797

In [17]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('\\')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
                i+=1
            else:
                x_test.append(img)
                y_test.append(z[1])
                i+=1

In [18]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [19]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [20]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (797, 3, 268, 182)
797 train samples
221 test samples


In [22]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

In [23]:
# Sample Resnet model
from torchvision import models

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.sigmoid(x)
        return x

In [25]:
model = ResNet()
#criterion = BCELoss()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
from statistics import mean

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)
        #print(output)
        #print(target)
        
        acc_list = []
        preds = torch.round(output)
        for i in range(len(preds)):
            result = 0
            denom = 0
            for j in range(len(classes)):
                if target[i][j] == 1 or preds[i][j] == 1:
                    denom += 1
                    if preds[i][j] == target[i][j]:
                        result+=1
            acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2%}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), mean(acc_list)))

def test():
    print('test')
    model.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        data, target = Variable(data, volatile=True).float(), Variable(target).float()
        output = model(data)
        
        acc_list = []
        preds = torch.round(output)
        for n in range(len(preds)):
            result = 0
            denom = 0
            for m in range(len(classes)):
                if target[n][m] == 1 or preds[n][m] == 1:
                    denom += 1
                    if preds[n][m] == target[n][m]:
                        result+=1
            acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAccuracy: {:.2%}'.format(test_loss.data.item()/i, mean(acc_list)))

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/797 (0%)]	Loss: 0.932713	Accuracy: 5.45%
Train Epoch: 0 [32/797 (4%)]	Loss: 0.892032	Accuracy: 14.99%
Train Epoch: 0 [64/797 (8%)]	Loss: 0.857787	Accuracy: 14.58%
Train Epoch: 0 [96/797 (12%)]	Loss: 0.824623	Accuracy: 9.38%
Train Epoch: 0 [128/797 (16%)]	Loss: 0.796992	Accuracy: 0.00%
Train Epoch: 0 [160/797 (20%)]	Loss: 0.775657	Accuracy: 0.00%
Train Epoch: 0 [192/797 (24%)]	Loss: 0.754115	Accuracy: 0.00%
Train Epoch: 0 [224/797 (28%)]	Loss: 0.739653	Accuracy: 0.00%
Train Epoch: 0 [256/797 (32%)]	Loss: 0.727126	Accuracy: 0.00%
Train Epoch: 0 [288/797 (36%)]	Loss: 0.717765	Accuracy: 0.00%
Train Epoch: 0 [320/797 (40%)]	Loss: 0.713577	Accuracy: 0.00%
Train Epoch: 0 [352/797 (44%)]	Loss: 0.706492	Accuracy: 0.00%
Train Epoch: 0 [384/797 (48%)]	Loss: 0.702458	Accuracy: 1.04%
Train Epoch: 0 [416/797 (52%)]	Loss: 0.701296	Accuracy: 2.08%
Train Epoch: 0 [448/797 (56%)]	Loss: 0.699306	Accuracy: 1.56%
Train Epoch: 0 [480/797 (60%)]	Loss: 0.697660	Accuracy: 4.69%
Train Epoch: 0

C:\Users\royal\AppData\Local\Temp\ipykernel_6776\3177561031.py:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.6964 
Accuracy: 33.33%
Train Epoch: 1 [0/797 (0%)]	Loss: 0.697166	Accuracy: 18.75%
Train Epoch: 1 [32/797 (4%)]	Loss: 0.695025	Accuracy: 18.75%
Train Epoch: 1 [64/797 (8%)]	Loss: 0.698803	Accuracy: 16.15%


KeyboardInterrupt: 

In [ ]:
import torchvision
import torch.nn as nn


model2 = torchvision.models.densenet201(pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 23)

criterion = BCEWithLogitsLoss()
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

In [ ]:
from statistics import mean

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)
        #print(output)
        #print(target)
        
        acc_list = []
        preds = torch.round(output)
        for i in range(len(preds)):
            result = 0
            denom = 0
            for j in range(len(classes)):
                if target[i][j] == 1 or preds[i][j] == 1:
                    denom += 1
                    if preds[i][j] == target[i][j]:
                        result+=1
            acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2%}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), mean(acc_list)))

def test():
    print('test')
    model.eval()
    test_loss = 0
    #correct = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        data, target = Variable(data, volatile=True).float(), Variable(target).float()
        output = model(data)
        
        acc_list = []
        preds = torch.round(output)
        for n in range(len(preds)):
            result = 0
            denom = 0
            for m in range(len(classes)):
                if target[n][m] == 1 or preds[n][m] == 1:
                    denom += 1
                    if preds[n][m] == target[n][m]:
                        result+=1
            acc_list.append(result/denom)
            
        loss = criterion(output, target)
        test_loss += loss
        #if loss==0:
            #correct+=1

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAccuracy: {:.2%}'.format(test_loss.data.item()/i, mean(acc_list)))

model.float()
#print(model)
for epoch in range(0, epochs):
    train(epoch)
    test()

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        human_loss= human_criterion(output, target)
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), human_loss.data.item()))

def test():
    print('test')
    model.eval()
    test_loss = 0
    correct = 0
    human_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        data, target = Variable(data, volatile=True).float(), Variable(target).float()
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss
        human_loss += human_criterion(output, target)
        if loss==0:
            correct+=1

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAverage abs_loss: {:.4f} \nGuessed 100% correct: {:.4f}\n'.format(test_loss.data.item()/i, human_loss.data.item()/i, correct))

model.float()
#print(model)
for epoch in range(0, epochs):
    train(epoch)
    test()